In [5]:
from CustomHamil import HubbardHamiltonian
from pyblock2.driver.core import DMRGDriver, SymmetryTypes
import numpy as np

L = 4
U = 2
omega = 5.0
lamb = 3.6
g = np.sqrt(lamb * omega / 2)
print("U =", U, "omega =", omega, "g =", g)
nbasis_fermion = L
nbasis_boson = L
nbcuts = [3] * L

driver = DMRGDriver(scratch="./tmp", symm_type=SymmetryTypes.SZ, n_threads=4)
b = driver.expr_builder()
driver.initialize_system(n_sites=nbasis_fermion+nbasis_boson, n_elec=L, spin=0, orb_sym=None)


driver.ghamil = HubbardHamiltonian(driver, driver.vacuum, driver.n_sites, driver.orb_sym, n_sites_fermion=nbasis_fermion, n_sites_boson=nbasis_boson, nbcuts=nbcuts)

for t, ops in zip([-1, 1, -1, 1], ["cd", "dc", "CD", "DC"]):
    b.add_term(ops, np.array([[i, i + 1] for i in range(L - 1)]).flatten(),
        [t] * (L - 1))

b.add_term("N", np.array([[i, ] for i in range(L)]).flatten(), [U] * L)

for t, ops in zip([g, g, g, g], ["cdE", "CDE", "cdF", "CDF"]):
    b.add_term(ops, np.array([[i, i, i + nbasis_fermion] for i in range(L)]).flatten(), [t] * L)

b.add_term("EF", np.array([[i, i] for i in range(nbasis_fermion, nbasis_boson + nbasis_fermion)]).flatten(), [omega] * L)

finilized_b = b.finalize(adjust_order=True)

mpo = driver.get_mpo(finilized_b, iprint=2)

ket = driver.get_random_mps(tag="GS", bond_dim=1000, nroots=1)

def run_dmrg(driver, mpo):
    bond_dims = [500] * 4 + [1000] * 4
    noises = [1e-4] * 4 + [1e-5] * 4 + [0]
    thrds = [1e-10] * 8
    return driver.dmrg(
        mpo,
        ket,
        n_sweeps=50,
        bond_dims=bond_dims,
        noises=noises,
        thrds=thrds,
        iprint=2,
    )

energies = run_dmrg(driver, mpo)


U = 2 omega = 5.0 g = 3.0
8

Build MPO | Nsites =     8 | Nterms =         36 | Algorithm = FastBIP | Cutoff = 1.00e-14
 Site =     0 /     8 .. Mmpo =     8 DW = 0.00e+00 NNZ =        8 SPT = 0.0000 Tmvc = 0.000 T = 0.009
 Site =     1 /     8 .. Mmpo =    10 DW = 0.00e+00 NNZ =       15 SPT = 0.8125 Tmvc = 0.000 T = 0.005
 Site =     2 /     8 .. Mmpo =    12 DW = 0.00e+00 NNZ =       17 SPT = 0.8583 Tmvc = 0.000 T = 0.011
 Site =     3 /     8 .. Mmpo =    10 DW = 0.00e+00 NNZ =       15 SPT = 0.8750 Tmvc = 0.000 T = 0.013
 Site =     4 /     8 .. Mmpo =     8 DW = 0.00e+00 NNZ =       11 SPT = 0.8625 Tmvc = 0.000 T = 0.007
 Site =     5 /     8 .. Mmpo =     6 DW = 0.00e+00 NNZ =        9 SPT = 0.8125 Tmvc = 0.000 T = 0.004
 Site =     6 /     8 .. Mmpo =     4 DW = 0.00e+00 NNZ =        7 SPT = 0.7083 Tmvc = 0.000 T = 0.012
 Site =     7 /     8 .. Mmpo =     1 DW = 0.00e+00 NNZ =        4 SPT = 0.0000 Tmvc = 0.000 T = 0.003
Ttotal =      0.063 Tmvc-total = 0.000 MPO bond dimensio

In [6]:
print('DMRG energy = %20.15f' % energies)

DMRG energy =  -11.135300766589626


In [3]:
import os
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["NUMEXPR_NUM_THREADS"] = "1"
os.environ["OMP_NUM_THREADS"] = "1"
import numpy as np
import pandas as pd
from pyscf import gto, scf
import pyqmc.api as pyq
import h5py
import matplotlib.pyplot as plt
from ase.data.pubchem import pubchem_atoms_search
import pyscf.pbc.tools.pyscf_ase as pyscf_ase
# Here we reset the files
for fname in ['mf.hdf5','optimized_wf.hdf5','vmc_data.hdf5','dmc.hdf5']:
    if os.path.isfile(fname):
        os.remove(fname)

def mean_field(chkfile):
    atoms =  pubchem_atoms_search(name='water')
    mol = gto.M(
        atom=pyscf_ase.ase_atoms_to_pyscf(atoms),
        basis="ccECP_cc-pVDZ", ecp="ccecp",
    )
    mf = scf.RHF(mol)
    mf.chkfile = chkfile
    mf.kernel()
mean_field("mf.hdf5")


converged SCF energy = -16.9321257267692


In [4]:
pyq.OPTIMIZE("mf.hdf5",           #Construct a Slater-Jastrow wave function from the pyscf output
             "optimized_wf.hdf5", #Store optimized parameters in this file.
             nconfig=100,         #Optimize using this many Monte Carlo samples/configurations
             max_iterations=50,   #10 optimization steps
             verbose=False)

df = pyq.read_opt("optimized_wf.hdf5")
df

energy  iteration     error              fname
0  -16.995435          0  0.009844  optimized_wf.hdf5
1  -17.092567          1  0.015378  optimized_wf.hdf5
2  -17.126003          2  0.015063  optimized_wf.hdf5
3  -17.136788          3  0.014270  optimized_wf.hdf5
4  -17.173230          4  0.010963  optimized_wf.hdf5
5  -17.157190          5  0.008360  optimized_wf.hdf5
6  -17.161536          6  0.012126  optimized_wf.hdf5
7  -17.179101          7  0.011219  optimized_wf.hdf5
8  -17.153705          8  0.009439  optimized_wf.hdf5
9  -17.176913          9  0.011703  optimized_wf.hdf5
10 -17.180690         10  0.007791  optimized_wf.hdf5
11 -17.178534         11  0.013845  optimized_wf.hdf5
12 -17.166321         12  0.014414  optimized_wf.hdf5
13 -17.169064         13  0.011798  optimized_wf.hdf5
14 -17.184935         14  0.006572  optimized_wf.hdf5
15 -17.173154         15  0.007665  optimized_wf.hdf5
16 -17.181303         16  0.007596  optimized_wf.hdf5
17 -17.201450         17  0.011875  optimized_wf.hdf5
18 -17.179054         18  0.006333  optimized_wf.hdf5
19 -17.166130         19  0.009248  optimized_wf.hdf5
20 -17.188134         20  0.008628  optimized_wf.hdf5
21 -17.179137         21  0.009721  optimized_wf.hdf5
22 -17.172567         22  0.012438  optimized_wf.hdf5
23 -17.173288         23  0.007844  optimized_wf.hdf5
24 -17.177211         24  0.011803  optimized_wf.hdf5
25 -17.182173         25  0.011279  optimized_wf.hdf5
26 -17.200919         26  0.006496  optimized_wf.hdf5
27 -17.200063         27  0.010234  optimized_wf.hdf5
28 -17.164814         28  0.006024  optimized_wf.hdf5
29 -17.169936         29  0.013400  optimized_wf.hdf5
30 -17.177148         30  0.009172  optimized_wf.hdf5
31 -17.183751         31  0.009199  optimized_wf.hdf5
32 -17.177959         32  0.007510  optimized_wf.hdf5
33 -17.183645         33  0.011211  optimized_wf.hdf5
34 -17.196401         34  0.006068  optimized_wf.hdf5
35 -17.169969         35  0.009706  optimized_wf.hdf5
36 -17.192133         36  0.013220  optimized_wf.hdf5
37 -17.193815         37  0.011887  optimized_wf.hdf5
38 -17.193270         38  0.011437  optimized_wf.hdf5
39 -17.191251         39  0.008593  optimized_wf.hdf5
40 -17.211837         40  0.008692  optimized_wf.hdf5
41 -17.196011         41  0.008744  optimized_wf.hdf5
42 -17.202631         42  0.004950  optimized_wf.hdf5
43 -17.179035         43  0.012733  optimized_wf.hdf5
44 -17.178431         44  0.009117  optimized_wf.hdf5
45 -17.200038         45  0.007622  optimized_wf.hdf5
46 -17.182341         46  0.005249  optimized_wf.hdf5
47 -17.186630         47  0.011989  optimized_wf.hdf5
48 -17.179395         48  0.011240  optimized_wf.hdf5
49 -17.182985         49  0.008401  optimized_wf.hdf5